# Informe Final de la Prueba Técnica 2

**Autor:** Isabel Castrillón Acosta

**Fecha:** 25-06-2025

**Descripción:** Este notebook presenta un informe detallado de la prueba técnica 2, que incluye la implementación de un chatbot legal utilizando modelos de lenguaje de Open AI. 



# 1. Explicación del Caso

En un consultorio legal, buscan una solución para utilizar la inteligencia artificial generativa (Generative AI), para permitir una asesoría automática a los clientes de la consultora legal. Y optimizar el tiempo de los abogados que asesoran a sus clientes frente a posibles demandas y los resultados de estas, al resolver dudas muy puntuales que no pueden ser resueltas solo con la historia de demandas y sentencias.

Quieren concentrarse en demandas relacionadas a redes sociales. 

La solución debe ser capaz de responder preguntas específicas sobre casos legales: 
- ¿Cuáles son las sentencias de 3 demandas? (las 3 demandas pueden ser elegidas por ut) 
- ¿De qué se trataron las 3 demandas anteriores? 
- ¿Cuál fue la sentencia del caso que habla de acoso escolar?
- ¿diga el detalle de la demanda relacionada con acoso escolar? 
- ¿existen casos que hablan sobre el PIAR, indique de que trataron los casos y 
cuáles fueron sus sentencias? 



# 2. Supuestos

- Se asume que los datos de los casos legales poseen solo demandas de redes sociales y que tienen toda la información necesaria para responder las preguntas planteadas. Ya que no se especifica que se deba buscar información adicional en otras fuentes.
Adicionalmente, al no tener conocimientos sobre las sentencias juridicas, se asume que las sentencias son correctas y que no se requiere una revisión de la terminología. 

# 3. Formas de resolver el caso y la opcion elegida

La mejor forma de darle acceso a personas que no tienen conocimientos técnicos a la información de los casos legales es a través de un chatbot, que utiliza un modelo de lenguaje para entender las preguntas y proporcionar respuestas relevantes basadas en los datos de los casos legales proporcionados.

- Para la creación del chatbot, se consideraron varias opciones de modelos de lenguaje preentrenados. Algunos opensource como Llama-2 o Huggin-face, y otros como los modelos de OpenAI.
- La opción elegida fue la implementación de un chatbot legal utilizando los modelos de OpenAI, ya que permite una interacción más natural y eficiente con los usuarios. Además, algunos de los otros modelos poseían incompatibilidades con el lenguaje en español, o con algunas librerías de Python que se utilizaron en el proyecto, lo que dificultaba su implementación.

# 4. Resultados del análisis de los datos y los modelos

## 4.1 Librerias

In [1]:
import pandas as pd
import faiss
import numpy as np
import os
from openai import OpenAI
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv


## 4.2 Carga de datos


In [2]:
df = pd.read_excel('C:/Users/Isabel/Documents/Dataknow-PT/Prueba tecnica 2/Datos/sentencias_pasadas.xlsx')
df.head()

,#,Relevancia,Providencia,Tipo,Fecha Sentencia,Tema - subtema,resuelve,sintesis
0,1,966965.0,T-185/22,NaN,2022-05-31,NaN,en nombre del pueblo y por mandato de la Const...,En este caso se formula la acción de tutela en...
1,3,963168.0,T-356/21,NaN,2021-10-15,ACCIÓN DE TUTELA PARA PROTEGER EL DERECHO A LA...,en nombre del pueblo y por mandato de la Const...,El peticionario considera que los accionantes ...
2,5,956201.0,T-351/22,NaN,2022-10-07,ACCIÓN DE TUTELA PARA PROTEGER EL DERECHO A LA...,"administrando justicia en nombre del Pueblo, y...",El periodista accionante acusa al abogado acci...
3,6,955889.0,T-246/21,NaN,2021-07-29,ACCION DE TUTELA PARA PROTEGER EL DERECHO A LA...,en nombre del pueblo y por mandato de la Const...,Se presenta la acción de tutela en contra de u...
4,7,955787.0,T-245A/22,NaN,2022-07-01,ACCION DE TUTELA-Inexistencia de hecho superad...,en nombre del pueblo y por mandato de la Const...,"El accionante, actuando en representación de s..."


## 4.3 Preparación de los datos


In [3]:
df['texto'] = df[['Providencia', 'Tipo', 'Fecha Sentencia', 'Tema - subtema', 'resuelve', 'sintesis']].astype(str).agg(' '.join, axis=1)
df['texto'].iloc[0]

'T-185/22 nan 2022-05-31 nan en nombre del pueblo y por mandato de la Constitución, RESUELVE PRIMERO.- REVOCAR las sentencias expedidas el 6 de enero de 2021, por el Juzgado Veinte Penal Municipal con Función de Control de Garantías de Bogotá, en primera instancia, y el 12 de febrero de 2021, por el Juzgado Cincuenta Penal del Circuito con Función de Conocimiento de Bogotá, en segunda instancia, mediante las cuales concedieron el amparo de los derechos fundamentales al buen nombre y a la honra de Enrique Vallejo Echeverri, Paula Camila Fonseca y la Clínica Envalle S.A.S. En su lugar, DECLARAR IMPROCEDENTE el amparo invocado por los accionantes, por las razones expuestas en la parte motiva de esta providencia. SEGUNDO.- Por Secretaría General, LÍBRENSE las comunicaciones de que trata el artículo 36 del Decreto 2591 de 1991. En este caso se formula la acción de tutela en contra de un particular, alegando la vulneración de los derechos fundamentales al buen nombre y honra, en virtud de la

## 4.4 Vectorización de los datos


Convertir los textos en embeddings:

In [4]:
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(df['texto'].tolist(), convert_to_numpy=True)

Guardar en base de datos vectorial con librería FAISS:


In [5]:
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

## 4.5 Función de búsqueda

In [6]:
def buscar_casos(query, top_k=3):
    query_emb = model.encode([query])
    D, I = index.search(query_emb, top_k)
    resultados = df.iloc[I[0]][['Providencia', 'texto']]
    return resultados


## 4.6 Crear chatbot

In [ ]:
load_dotenv()
client = OpenAI(api_key= os.getenv('OPENAI_API_KEY'))  

def consulta_chatbot_openai(pregunta, top_k=3):
    contexto = buscar_casos(pregunta, top_k=top_k).head(3)
    contexto_completo = ""
    for i, fila in contexto.iterrows():
        contexto_completo += f"Caso {i+1}:\n{fila['texto']}\n\n"

    prompt = f"""Actúa como un asesor legal que explica de manera sencilla para alguien sin conocimientos jurídicos o técnicos.

Contexto extraído de casos reales:
{contexto_completo}

Pregunta: {pregunta}
Respuesta:"""

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.5,
        max_tokens=500
    )

    return response.choices[0].message.content.strip()



## 4.7 Pruebas del chatbot

In [8]:
pregunta = "Elige tres demandas, las demandas las puedes elegir tu. ¿Cuáles son las sentencias de 3 demandas?"
respuesta = consulta_chatbot_openai(pregunta)
print(respuesta)

1. Caso 196: La sentencia declara la Ley 2300 de 2023 como EXEQUIBLE, únicamente por el cargo analizado en la demanda de inconstitucionalidad.
2. Caso 141: La sentencia CONFIRMA el rechazo de la demanda de inconstitucionalidad contra el artículo 189 de la Ley 906 de 2004.
3. Caso 271: La sentencia declara INEXEQUIBLES ciertos numerales del Estatuto Tributario, específicamente el inciso 2° del numeral 2° del artículo 905 y los numerales 4° y 5° del artículo 908, entre otros, de acuerdo a la demanda presentada.


In [9]:
pregunta = "¿De qué se trataron las 3 demandas anteriores?"
respuesta = consulta_chatbot_openai(pregunta)
print(respuesta)

Las tres demandas anteriores están relacionadas con temas legales y constitucionales en Colombia. En el primer caso, se cuestiona la diferencia de trato injustificada entre distintos grupos que ejercen profesiones liberales en el régimen simple de tributación. En el segundo caso, se analiza la vulneración de derechos fundamentales por parte de una empresa que ofrecía préstamos de dinero y realizaba prácticas abusivas de cobranza. En el tercer caso, se hace un seguimiento a la sentencia T-302 de 2017, que declaró un estado de cosas inconstitucional en relación con el goce efectivo de los derechos fundamentales de la comunidad Wayuu en La Guajira.


In [10]:
pregunta = "¿Cuál fue la sentencia del caso que habla de acoso escolar?"
respuesta = consulta_chatbot_openai(pregunta)
print(respuesta)

La sentencia del caso que habla de acoso escolar es la T-252/23.


In [11]:
pregunta = "¿diga el detalle de la demanda relacionada con acoso escolar?"
respuesta = consulta_chatbot_openai(pregunta)
print(respuesta)

En el caso 276, la demanda está relacionada con la prohibición por parte de la Universidad C de que la actora ingresara al salón de clases con su perro de apoyo emocional. La actora argumentó que necesitaba la presencia de su perro para ayudar en el tratamiento de sus problemas de salud mental. La universidad, por su parte, exigió a la demandante presentar documentos relacionados con su historia clínica y el carnet de vacunación de su mascota para permitir su ingreso al campus. La demandante consideró que esta medida era discriminatoria y afectaba su derecho a la educación y a la igualdad. Como resultado, la Corte Constitucional ordenó a la Universidad C presentar excusas privadas a la demandante, permitirle retomar sus estudios y ajustar sus protocolos para garantizar el acceso de perros de apoyo emocional en sus instalaciones.


In [12]:
pregunta = "¿existen casos que hablan sobre el PIAR, indique de que trataron los casos y cuáles fueron sus sentencias?"
respuesta = consulta_chatbot_openai(pregunta)
print(respuesta)

No se menciona ningún caso específico sobre el PIAR en los contextos proporcionados. El PIAR se refiere al Procedimiento de Insolvencia de la Persona Natural No Comerciante, que es un mecanismo legal para que las personas que no son comerciantes y se encuentran en situación de insolvencia puedan acogerse a un proceso de negociación de deudas. Si necesitas información específica sobre casos relacionados con el PIAR, te recomendaría buscar en bases de datos jurídicas o consultar con un abogado especializado en derecho de insolvencia.


## 4.8 Conclusiones

El chatbot es capaz de responder preguntas específicas sobre casos legales relacionados con redes sociales, utilizando los datos proporcionados.

Podría optimizarse aún más la precisión de las respuestas, ajustando los parámetros del modelo o utilizando técnicas adicionales de procesamiento de lenguaje natural.

Sin embargo, es una herramienta útil para proporcionar información rápida y precisa a los usuarios sin necesidad de conocimientos técnicos avanzados.

# 5. Futuras mejoras

- Teniendo en cuenta que el modelo de OpenAI tiene un costo por token, se podrían explorar alternativas de modelos de lenguaje más económicos o incluso entrenar un modelo propio si se dispone de los recursos necesarios para ahorrar en costos a largo plazo.


# 6. Apreciaciones y comentarios del caso (opcional)